In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from WelfareMeasures import *
from functions_njit import tax_rate_fct
from bernoulli_distribution import Bernoulli
from help_functions_non_njit import *
from graph_format import *
# load local model file and initialize model class
from model import ModelClass
from scipy.optimize import minimize

# Load original model

In [2]:
model = ModelClass() # set a few number of periods.

par = model.par
sol = model.sol
sim = model.sim
par.opt_tol = 1e-6
par.opt_maxiter = 1000

%time model.solve(do_print=False)
%time model.simulate()

CPU times: total: 13min 13s
Wall time: 4min 23s
CPU times: total: 22.3 s
Wall time: 34.6 s


In [ ]:
model.sol.V[20,5,5,5,20,1,1]
np.mean(model.sim.c[:,0])

np.float64(-18.125116056877868)

In [8]:
np.mean(model.sim.c[:,0])

np.float64(171489.54900989175)

# Counter factual

In [ ]:
# Change parameters and re-solve

total_effect_list = []

for age in range(35, 50):

    theta = [age]
    theta_names = ['retirement_age']

    og_model, new_model =  make_new_model(model, theta, theta_names, do_print = False)

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.sum(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
    total_margin_new = np.sum(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
    total_margin = (total_margin_new-total_margin_og)/total_margin_og

    total_effect_list.append([total_margin_og, total_margin_new, total_margin])

In [ ]:
plot_total_effect_with_trend(total_effect_list, title=None, save_title="ret_age_increase_effect.png")

In [ ]:
ret_age = 49
increasing_RPA = []

for first_age in range(30, ret_age + 1):
    early_age_lag = ret_age- first_age-4

    theta = [ret_age, first_age, early_age_lag]
    theta_names = ['retirement_age', 'first_retirement', 'early_benefits_lag', ]

    og_model, new_model = make_new_model(model, theta, theta_names, do_print=False)
    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.nanmean(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
    total_margin_new = np.nanmean(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
    total_margin = (total_margin_new - total_margin_og)

    increasing_RPA.append([ret_age, first_age, early_age_lag+ first_age, total_margin, consumption_eq])
    print("Done with first_age", first_age)

np.savetxt('increasing_RPA.txt', increasing_RPA)

In [ ]:
first_retirement = 30
early_age_lag = 1
increasing_SPA_fix_RPA_EPA = []

for ret_age in range(first_retirement +early_age_lag, 50):

    theta = [ret_age, first_age, early_age_lag]
    theta_names = ['retirement_age', 'first_retirement', 'early_benefits_lag', ]

    og_model, new_model = make_new_model(model, theta, theta_names, do_print=False)
    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.nanmean(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
    total_margin_new = np.nanmean(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
    total_margin = (total_margin_new - total_margin_og)

    increasing_SPA_fix_RPA_EPA.append([ret_age, first_age, early_age_lag+ first_age, total_margin, consumption_eq])
    print("Done with first_age", ret_age)

np.savetxt('increasing_SPA_fix_RPA_EPA.txt', increasing_SPA_fix_RPA_EPA)

In [ ]:
first_retirement = 30
ret_age = 49
increasing_EPA_fix_SPA_RPA = []

for early_age_lag in range(0, ret_age-first_retirement):

    theta = [ret_age, first_age, early_age_lag]
    theta_names = ['retirement_age', 'first_retirement', 'early_benefits_lag', ]

    og_model, new_model = make_new_model(model, theta, theta_names, do_print=False)
    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.nanmean(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
    total_margin_new = np.nanmean(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
    total_margin = (total_margin_new - total_margin_og)

    increasing_EPA_fix_SPA_RPA.append([ret_age, first_age, early_age_lag+ first_age, total_margin, consumption_eq])
    print("Done with first_age", early_age_lag)

np.savetxt('increasing_SPA_fix_RPA_EPA.txt', increasing_EPA_fix_SPA_RPA)

In [ ]:
total_effect_grid = []

count = 0

for ret_age in range(35, 50):
    for first_age in range(ret_age - 5, ret_age + 1):
        for early_age_lag in range(ret_age - first_age + 1):

            theta = [ret_age, first_age, early_age_lag]
            theta_names = ['retirement_age', 'first_retirement', 'early_benefits_lag', ]

            og_model, new_model = make_new_model(model, theta, theta_names, do_print=False)
            consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

            pi_cum = np.cumprod(og_model.par.pi)

            total_margin_og = np.nanmean(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
            total_margin_new = np.nanmean(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
            total_margin = (total_margin_new - total_margin_og)

            total_effect_grid.append([ret_age, first_age, early_age_lag, total_margin, consumption_eq])

            count += 1

            print("Done with iteration", count)


In [ ]:
total_effect_grid

In [ ]:
np.savetxt('total_effect_grid.txt', total_effect_grid)

In [ ]:
# Change parameters and re-solve

total_effect_list_new = []

for early_lag in range(5):

    temp_list = []
    for age in range(35, 50):

        theta = [age, early_lag]
        theta_names = ['retirement_age', 'early_benefits_lag']

        og_model, new_model =  make_new_model(model, theta, theta_names, do_print = False)

        pi_cum = np.cumprod(og_model.par.pi)

        total_margin_og = np.sum(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
        total_margin_new = np.sum(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
        total_margin = (total_margin_new-total_margin_og)/total_margin_og

        temp_list.append([total_margin_og, total_margin_new, total_margin])

    total_effect_list_new.append(temp_list)


In [ ]:
total_effect_efter = np.array(total_effect_list_new)

# np.savetxt('Data/total_effect_against_early.txt', total_effect_efter, fmt='%.6f')

np.save('Data/total_effect_early_ret.npy', total_effect_efter)

In [ ]:
plt.plot(total_effect_efter[0][:, 2])
plt.plot(total_effect_efter[1][:, 2])
plt.plot(total_effect_efter[2][:, 2])
plt.plot(total_effect_efter[3][:, 2])
plt.plot(total_effect_efter[4][:, 2])

In [ ]:
# Change parameters and re-solve

total_effect_list_ages = []

for age in range(35, 50):

    theta = [age]
    theta_names = ['retirement_age']

    og_model, new_model =  make_new_model(model, theta, theta_names, do_print = False)

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.sum(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement], axis = 0)
    total_margin_new = np.sum(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement], axis = 0)
    # total_margin = (total_margin_new-total_margin_og)/total_margin_og

    total_effect_list_ages.append([total_margin_og, total_margin_new])

In [ ]:
arr = np.array(total_effect_list_ages)
df = pd.DataFrame(arr.reshape(arr.shape[0], -1))  # shape: (15, 110)

In [ ]:
plt.plot((df.iloc[:, 55::].sum(axis=1) - df.iloc[:, :55].sum(axis=1)) / df.iloc[:, :55].sum(axis=1))

In [ ]:
plt.plot((df.iloc[:, 85:].sum(axis=1) - df.iloc[:, 30:55].sum(axis=1)) / df.iloc[:, 30:55].sum(axis=1))
plt.plot((df.iloc[:, 55:85].sum(axis=1) - df.iloc[:, 0:30].sum(axis=1)) / df.iloc[:, 0:30].sum(axis=1))

In [ ]:
for alder in range(0,55):

    plt.plot(df[55+alder] - df[0+alder])

In [ ]:
assert False

# Compare models 

In [ ]:
# Compare models 
consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= True, the_method = 'brentq')
intensive_margin, extensive_margin, total_margin, intensive_margin_age, extensive_margin_age, total_margin_age  = labor_elasticity(og_model, new_model)


plot_labor_margins_by_age(
    intensive_age=intensive_margin_age,
    extensive_age=extensive_margin_age,
    total_age=total_margin_age,
    avg_intensive=extensive_margin,
    avg_extensive=intensive_margin,
    avg_total=total_margin,
    age_start=par.start_age,
    title_prefix="Labor Supply — ",
    save_title="counterfactual_labor_margins_by_age.png"
)

In [ ]:
con_eq = analytical_consumption_equivalence_indi(og_model, new_model)

In [ ]:
plot_event_histogram(con_eq[:], 'Consumption Equivalence', 'Consumption Equivalence Distribution',
                         label1="Event 1",
                         figsize=(10, 6), bins=100, save_title='consumption_equivalence_indi')

In [ ]:
plt.hist(con_eq[:], bins=100)

In [ ]:
np.round(np.mean(con_eq[:])*100,1)

In [ ]:
np.round(np.mean(con_eq[:])*100,1)

# comparison of means

In [ ]:
time = np.arange(par.T)

# Data for selected summary variables
summary_vars = ["Wages (w)", "Assets (a)", "Consumption (c)", "Illiquid Savings (s)"]
simulated_data_new = {
    "Wages (w)": np.mean(new_model.sim.w[:], axis=0),
    "Assets (a)": np.mean(new_model.sim.a[:], axis=0),
    "Consumption (c)": np.mean(new_model.sim.c[:], axis=0),
    "Illiquid Savings (s)": np.mean(new_model.sim.s[:], axis=0),
}
simulated_data_og = {
    "Wages (w)": np.mean(og_model.sim.w[:], axis=0),
    "Assets (a)": np.mean(og_model.sim.a[:], axis=0),
    "Consumption (c)": np.mean(og_model.sim.c[:], axis=0),
    "Illiquid Savings (s)": np.mean(og_model.sim.s[:], axis=0),
}

plot_comparison_single_panel(simulated_data_og, simulated_data_new, summary_vars, time, save_title="counterfactual_summary_vars.png")


In [ ]:

# Full variable set
data_dict_new = {
    "Human Capital (k)": np.mean(new_model.sim.k[:], axis=0),
    "Wages (w)": np.mean(new_model.sim.w[:], axis=0),
    "Hours (hours)": np.nanmean(np.where(new_model.sim.ex == 1, new_model.sim.h, np.nan), axis=0),
    "Assets (a)": np.mean(new_model.sim.a[:], axis=0),
    "Consumption (c)": np.mean(new_model.sim.c[:], axis=0),
    "Illiquid Savings (s)": np.mean(new_model.sim.s[:], axis=0),
    "Extensive Margin (ex)": np.mean(new_model.sim.ex[:], axis=0),
    "Tax rate (tax_rate)": np.mean(new_model.sim.tax_rate[:,:], axis=0),
    "Wages after taxes": np.mean((1 - new_model.sim.tax_rate[:,:]) * new_model.sim.w[:,:], axis=0),
}

data_dict_og = {
    "Human Capital (k)": np.mean(og_model.sim.k[:], axis=0),
    "Wages (w)": np.mean(og_model.sim.w[:], axis=0),
    "Hours (hours)": np.nanmean(np.where(og_model.sim.ex == 1, og_model.sim.h, np.nan), axis=0),
    "Assets (a)": np.mean(og_model.sim.a[:], axis=0),
    "Consumption (c)": np.mean(og_model.sim.c[:], axis=0),
    "Illiquid Savings (s)": np.mean(og_model.sim.s[:], axis=0),
    "Extensive Margin (ex)": np.mean(og_model.sim.ex[:], axis=0),
    "Tax rate (tax_rate)": np.mean(og_model.sim.tax_rate[:,:], axis=0),
    "Wages after taxes": np.mean((1 - og_model.sim.tax_rate[:,:]) * og_model.sim.w[:,:], axis=0),
}

data_dict_diff = {
    "Human Capital (k)": np.mean(new_model.sim.k[:], axis=0) - np.mean(og_model.sim.k[:], axis=0),
    "Wages (w)": np.mean(new_model.sim.w[:], axis=0) - np.mean(og_model.sim.w[:], axis=0),
    "Hours (hours)": np.nanmean(np.where(new_model.sim.ex == 1, new_model.sim.h, np.nan), axis=0)
                    - np.nanmean(np.where(og_model.sim.ex == 1, og_model.sim.h, np.nan), axis=0),
    "Assets (a)": np.mean(new_model.sim.a[:], axis=0) - np.mean(og_model.sim.a[:], axis=0),
    "Consumption (c)": np.mean(new_model.sim.c[:], axis=0) - np.mean(og_model.sim.c[:], axis=0),
    "Illiquid Savings (s)": np.mean(new_model.sim.s[:], axis=0) - np.mean(og_model.sim.s[:], axis=0),
    "Extensive Margin (ex)": np.mean(new_model.sim.ex[:], axis=0) - np.mean(og_model.sim.ex[:], axis=0),
    "Tax rate (tax_rate)": np.mean(new_model.sim.tax_rate[:,:], axis=0) - np.mean(og_model.sim.tax_rate[:,:], axis=0),
    "Wages after taxes": (
        np.mean((1 - new_model.sim.tax_rate[:,:]) * new_model.sim.w[:,:], axis=0)
        - np.mean((1 - og_model.sim.tax_rate[:,:]) * og_model.sim.w[:,:], axis=0)
    ),
}

plot_comparison_grid(data_dict_og, data_dict_new, time, title="Original vs New Simulations", save_title="counterfactual_full_vars.png")
plot_difference_grid(data_dict_diff, time, title="Difference between New and Original Simulations", save_title="counterfactual_diff_full_vars.png")


In [ ]:
# 1. Find last time ex == 1 for each individual
last_working = np.full(og_model.sim.ex.shape[0], np.nan)  # initialize

for i in range(og_model.sim.ex.shape[0]):
    working = np.where(og_model.sim.ex[i, :] == 1)[0]  # find where ex=1
    if working.size > 0:
        last_working[i] = working[-1]  # last time they worked (take last index)

# 1. Find last time ex == 1 for each individual
last_working_new = np.full(new_model.sim.ex.shape[0], np.nan)  # initialize

for i in range(new_model.sim.ex.shape[0]):
    working_new = np.where(new_model.sim.ex[i, :] == 1)[0]  # find where ex=1
    if working_new.size > 0:
        last_working_new[i] = working_new[-1]  # last time they worked (take last index)


In [ ]:
plot_event_bar_series(
    values1=last_working,
    values2=last_working_new,
    xlabel="Retirement Age",
    title="Timing of Unofficial Retirement",
    label1="Old",
    label2=f'New ({theta_names[0]} = {theta[0]})',
    save_title="counterfactual_retirement_age.png",
)


In [ ]:
# 1. Find first time e == 2 for each individual
first_retirement = np.full(og_model.sim.e.shape[0], np.nan)  # initialize

for i in range(og_model.sim.e.shape[0]):
    retired = np.where(og_model.sim.e[i, :] == 2)[0]  # find where e=2
    if retired.size > 0:
        first_retirement[i] = retired[0]  # first time they retire

first_retirement_new = np.full(new_model.sim.e.shape[0], np.nan)  # initialize

for i in range(new_model.sim.e.shape[0]):
    retired_new = np.where(new_model.sim.e[i, :] == 2)[0]  # find where e=2
    if retired_new.size > 0:
        first_retirement_new[i] = retired_new[0]  # first time they retire
plot_event_bar_series(
    values1=first_retirement,
    values2=first_retirement_new,
    xlabel="Official Retirement Age",
    title="Timing of Official Retirement",
    label1="Old",
    label2=f'New ({theta_names[0]} = {theta[0]})',
    save_title="counterfactual_official_retirement_age.png",
)


In [ ]:
og_ret_counts = np.sum(og_model.sim.ret_flag[:, :], axis=0)
new_ret_counts = np.sum(new_model.sim.ret_flag[:, :], axis=0)
plot_bar_series_comparison(
    og_ret_counts,
    new_ret_counts,
    label1="Old",
    label2=f"New (retirement_age = {theta[0]})",
    title="Voluntary Retirements per Period",
    xlabel="Age (Period)",
    normalize=True,
    save_title="counterfactual_retirement_counts.png",
)


In [ ]:
plot_event_histogram(
    values1=consumption_replacement_rate_fct(og_model),
    values2=consumption_replacement_rate_fct(new_model),
    xlabel="Age (Period)",
    title="Consumption Replacement Rate",
    label1="Old",
    label2=f'New ({theta_names[0]} = {theta[0]})',
    bins=30,
    save_title="counterfactual_consumption_replacement_histogram.png",
)

In [ ]:
plt.plot(og_model.par.p_e_0)
plt.plot(new_model.par.p_e_0)

In [ ]:
plt.plot(og_model.par.p_e_1)
plt.plot(new_model.par.p_e_1)

In [ ]:
plt.plot(og_model.par.p_e_2)
plt.plot(new_model.par.p_e_2)

In [ ]:
plt.plot(np.mean(new_model.sim.ex, axis=0)-np.mean(og_model.sim.ex, axis=0))

In [ ]:
np.mean(new_model.sim.ex, axis=0)

# Comparison of retirement age

# replacement rates

In [ ]:
assert False

# Optimal retirement payments according to welfare

In [ ]:
def objective_func(tau_values, model):
    time = np.arange(par.T)

    constant, beta1, beta2 = tau_values

    tau = np.maximum((constant + beta1*time + beta2*time**2) / 100, 0)

    theta = [tau]
    theta_names = ['tau']

    og_model, new_model =  make_new_model(model, theta, theta_names, do_print = False)

    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    print(consumption_eq, tau_values)
    return -consumption_eq

tau_values_init = np.array([1.35e+01, 8.00e-02, 5.00e-04])

bounds  = [(0,  30), 
           (-0.5,  0.5), 
           (-0.005,  0.005)]

res = minimize(objective_func, tau_values_init, args=(model, ),  
               method='Nelder-Mead', 
               bounds=bounds,
               options={'maxiter': 500})

print(res.x)

# Optimal retirement payments according to welfare conditioning on pension being the same

In [ ]:
def objective_func(tau_values, model):
    time = np.arange(par.T)

    constant, beta1, beta2 = tau_values

    tau = np.maximum((constant + beta1*time + beta2*time**2) / 100, 0)

    theta = [tau]
    theta_names = ['tau']

    og_model, new_model =  make_new_model(model, theta, theta_names, do_print = False)

    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    if np.mean(og_model.sim.s[:, 35]) > np.mean(new_model.sim.s[:, 35]):
        consumption_eq = -9999.9

    print(consumption_eq, tau_values)
    return -consumption_eq

tau_values_init = np.array([ 2.0, 0.4,  0.001])

bounds  = [(0,  30), 
           (-0.5,  0.5), 
           (-0.005,  0.005)]

res = minimize(objective_func, tau_values_init, args=(model, ),  
               method='Nelder-Mead', 
               bounds=bounds,
               options={'maxiter': 500})

print(res.x)

In [ ]:
# constant, beta1, beta2 = np.array([ 1.46922503e+01, -2.31206543e-02,  6.27520107e-04]) # Current optimal giving 0.0065521344388923845 consumption eq.
constant, beta1, beta2 = np.array([ 1.48536631e+01, -4.18610966e-02,  6.47243901e-04]) # Test start


time = np.arange(par.T)

tau = np.maximum((constant + beta1*time + beta2*time**2) / 100, 0)

plt.plot(tau)
plt.plot(par.tau)
plt.show()


In [ ]:
theta = [tau]
theta_names = ['tau']

og_model, new_model =  make_new_model(model, theta, theta_names, do_print = False)

In [ ]:
a_dict = {
    'hours': [np.nanmean(np.where(model.sim.ex == 1, model.sim.h, np.nan), axis=0)[:40],
              moments['hours']],
    'extensive': [np.mean(model.sim.ex, axis=0)[:40],
                  moments['extensive']],
    'illiquid': [np.clip(np.mean(model.sim.s, axis=0), 0, None),
                 moments['savings']],
    'liquid': [np.mean(model.sim.a, axis=0),
               moments['assets']]
}

# set up 2x2 axes
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for ax, (key, (sim, emp)) in zip(axes, a_dict.items()):
    x_vals = np.arange(len(emp)) + par.start_age

    # real data: solid grey
    ax.plot(x_vals, emp,
            color='grey',
            linestyle='-',
            linewidth=2,
            label=f"Real {key.capitalize()}")

    # simulated data: darker dashed
    ax.plot(x_vals, sim,
            color='black',
            linestyle='--',
            linewidth=2,
            label=f"Simulated {key.capitalize()}")

    ax.set_xlabel("Age")
    ax.set_ylabel(key.capitalize())
    ax.set_title(key.capitalize())
    ax.grid(True)
    ax.legend()

fig.tight_layout()
plt.show()
